In [2]:
import pandas as pd, numpy as np
import os
from glob import glob as g

In [14]:
def split_dataframe(df, length):
    # List to hold the split DataFrames
    split_dfs = []
    
    # Calculate the number of splits needed
    num_splits = len(df) // length + (1 if len(df) % length != 0 else 0)
    
    for i in range(num_splits):
        # Calculate start and end indices for the current split
        start_idx = i * length
        end_idx = start_idx + length
        
        # Slice the DataFrame and add it to the list
        split_dfs.append(df.iloc[start_idx:end_idx])
    print(f"Created {len(split_dfs)} chunks.")
    return split_dfs

In [4]:
## set source 
source = r'C:\Users\AbhishekSrivastava\Desktop\Workflow\Python\Dev\Medical Devices Registry\MDR Scripts\AR'
file = "\ANMAT_AR Full Extract_260124.xlsx"

In [15]:
payload = pd.read_excel(source+file)
payload.shape

In [16]:
chunks = split_dataframe(payload, 750)

Created 41 chunks.


In [18]:
for chunk in chunks:
    print(chunk.shape)

(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(750, 11)
(419, 11)


In [20]:
chunks[0].head(2)

,Unnamed: 0,Trámite,Fecha Fin,Razón Social,Nombre,Marca,Modelo/s,PM,Expediente,Dec/Cert,Disposición
0,0,Registro Clase III-IV,25-01-2024,DROGUERIA MARTORANI S.A.,Apósito transparente I.V. reforzado con glucon...,IHT,"Oper film ®protect CHG 7 x 8,5 cm (Ref: 003400...",928-552,1-0047-3110-005060-23-9,NaN,NaN
1,0,Registro Clase III-IV,25-01-2024,TESIS SRL,Válvula cardíaca biológica,Meril,Aórticos:\nDDL19A; DDL21A; DDL23A; DDL25A; DDL...,867-44,1-0047-3110-005087-23-3,NaN,NaN


In [ ]:
## Translate one by one

In [23]:
cols = chunks[0].columns
cols

Index(['Unnamed: 0', 'Trámite', 'Fecha Fin', 'Razón Social', 'Nombre', 'Marca',
       'Modelo/s', 'PM', 'Expediente', 'Dec/Cert', 'Disposición'],
      dtype='object')

In [22]:
from googletrans import Translator
translator = Translator()
from functools import lru_cache

@lru_cache(maxsize=None)
def translate_to_en(text):
    if pd.notna(text):
        translated_text = translator.translate(str(text), dest='en').text
        return translated_text
    else:
        return text

In [25]:
en_cols = [translate_to_en(x) for x in cols]
en_cols

['Unnamed: 0',
 'Procedure',
 'End date',
 'Business name',
 'Name',
 'Brand',
 'Models',
 'PM',
 'Office hour',
 'Dec/Cert',
 'Provision']

In [34]:
len(chunks[0]['Marca'].unique())

535

In [35]:
len(chunks[0]['Nombre'].unique())

702

In [30]:
translated_chunks = []

In [29]:
import time

In [40]:

for i, chunk in enumerate(chunks):
    if i < 10:
        continue
    else:
        print(f"Translating chunk: {i}")
        _s = time.time()
        translated_chunk = chunk.copy()
        translated_chunk.columns = en_cols

        translated_chunk['Business name'] = translated_chunk['Business name'].map(lambda x: translate_to_en(x))
        translated_chunk['Name'] = translated_chunk['Name'].map(lambda x: translate_to_en(x))
        translated_chunk['Models'] = translated_chunk['Models'].map(lambda x: translate_to_en(x))
        translated_chunk['Brand'] = translated_chunk['Brand'].map(lambda x: translate_to_en(x))
        translated_chunks.append(translated_chunk)
        _e = time.time()
        print(f"Time Elasped: {round(_e - _s, 2)} seconds.")

Translating chunk: 10
Time Elasped: 171.85 seconds.
Translating chunk: 11
Time Elasped: 392.57 seconds.
Translating chunk: 12
Time Elasped: 388.85 seconds.
Translating chunk: 13


KeyboardInterrupt: 

In [41]:
len(translated_chunks)

13

In [42]:
for i, chunk in enumerate(translated_chunks):
    chunk.to_excel(f"preprcoess dump/AR_Translated/ar_chunk {i}.xlsx")

In [44]:
## Load all ttranslted chunks
files = g(os.path.join("preprcoess dump/AR_Translated","*.xlsx"))

In [49]:
cols_to_pick = ['Procedure',
 'End date',
 'Business name',
 'Name',
 'Brand',
 'Models']

cols_to_pick

['Procedure', 'End date', 'Business name', 'Name', 'Brand', 'Models']

In [53]:
export = []
for file in files:
    print(file)
    data = pd.read_excel(file)
    chunk = data[cols_to_pick]
    export.append(chunk)

preprcoess dump/AR_Translated\ar_chunk 0.xlsx
preprcoess dump/AR_Translated\ar_chunk 1.xlsx
preprcoess dump/AR_Translated\ar_chunk 10.xlsx
preprcoess dump/AR_Translated\ar_chunk 11.xlsx
preprcoess dump/AR_Translated\ar_chunk 12.xlsx
preprcoess dump/AR_Translated\ar_chunk 13.xlsx
preprcoess dump/AR_Translated\ar_chunk 14.xlsx
preprcoess dump/AR_Translated\ar_chunk 15.xlsx
preprcoess dump/AR_Translated\ar_chunk 16.xlsx
preprcoess dump/AR_Translated\ar_chunk 17.xlsx
preprcoess dump/AR_Translated\ar_chunk 18.xlsx
preprcoess dump/AR_Translated\ar_chunk 19.xlsx
preprcoess dump/AR_Translated\ar_chunk 2.xlsx
preprcoess dump/AR_Translated\ar_chunk 20.xlsx
preprcoess dump/AR_Translated\ar_chunk 21.xlsx
preprcoess dump/AR_Translated\ar_chunk 22.xlsx
preprcoess dump/AR_Translated\ar_chunk 23.xlsx
preprcoess dump/AR_Translated\ar_chunk 24.xlsx
preprcoess dump/AR_Translated\ar_chunk 25.xlsx
preprcoess dump/AR_Translated\ar_chunk 26.xlsx
preprcoess dump/AR_Translated\ar_chunk 27.xlsx
preprcoess dump/

In [54]:
anmat_ar_en = pd.concat(export)

In [55]:
anmat_ar_en.shape

(30419, 6)

In [56]:
anmat_ar_en.to_excel("ANMAT AR MDR Jan24.xlsx")